# UGT2B7 Isoform Expression Analysis

In [ ]:
install.packages("htmltools")
install.packages("BiocManager")
BiocManager::install("DESeq2")

In [ ]:
library(htmltools)
library( "DESeq2" )
library(ggplot2)
library(magrittr)
library(tidyr)

In [ ]:
# CRAN
library(dplyr)
library(tidyr)
library(ggplot2)
theme_set(theme_bw(base_size=14) + theme(strip.background = element_blank()))

# Bioconductor
library(DESeq2)
library(airway)

In [ ]:
countData <- read.csv("../data/input/pcawg.rnaseq.transcript.UGT2B7.expr.counts.csv", header = TRUE, sep = ",")
head(countData)

In [ ]:
metaData <- read.csv('../data/input/ugt2b7_expr_metadata.csv', header = TRUE, sep = ",")
head(metaData)

In [ ]:
dds <- DESeqDataSetFromMatrix(countData=countData, 
                              colData=metaData, 
                              design=~exp, tidy = TRUE)

In [ ]:
dds <- estimateSizeFactors(dds)
dds <- estimateDispersionsGeneEst(dds)
dispersions(dds) <- mcols(dds)$dispGeneEst
dds <- nbinomWaldTest(dds)

In [ ]:
res <- results(dds)

In [ ]:
res

In [ ]:
pdf(file = "output/plot/exp_UGT2B7.pdf",   # The directory you want to save the file in
    width = 6, # The width of the plot in inches
    height = 4)

par(mfrow=c(1,3))

plotCounts(dds, gene="ENST00000508661.1", intgroup="exp")
plotCounts(dds, gene="ENST00000305231.7", intgroup="exp")
plotCounts(dds, gene="ENST00000502942.1", intgroup="exp")

dev.off()

In [ ]:
goi <- c('ENST00000508661.1', 'ENST00000305231.7', 'ENST00000502942.1')

In [ ]:
# tcounts <- t(log2((counts(dds[goi, ], normalized=TRUE, replaced=FALSE)+.5))) %>%
#   merge(colData(dds), ., by="row.names") %>%
#   gather(gene, expression, (ncol(.)-length(goi)+1):ncol(.))

In [ ]:
tcounts <- t((counts(dds[goi, ], normalized=TRUE, replaced=FALSE))) %>%
  merge(colData(dds), ., by="row.names") %>%
  gather(gene, expression, (ncol(.)-length(goi)+1):ncol(.))

In [ ]:
tcounts %>% 
  select(Row.names, exp, gene, expression) %>% 
  head

In [ ]:
pdf(file = "output/plot/exp_UGT2B7_box_linear.pdf",   # The directory you want to save the file in
    width = 6, # The width of the plot in inches
    height = 4)

ggplot(tcounts, aes(exp, expression)) + 
  geom_boxplot() + 
  facet_wrap(~gene, scales="free_y") + 
  labs(x="rRE Expanded", 
       y="Expression (log normalized counts)")



dev.off()

In [ ]:
pdf(file = "output/plot/exp_UGT2B7_violin.pdf",   # The directory you want to save the file in
    width = 6, # The width of the plot in inches
    height = 4)

ggplot(tcounts, aes(exp, expression)) + 
  geom_violin() + 
  facet_wrap(~gene, scales="free_y") + 
  labs(x="rRE Expanded", 
       y="Expression (log normalized counts)")

dev.off()